<a href="https://colab.research.google.com/github/GDharan10/Analysis1_RetailSalsesForecast/blob/main/RetailSalsesForecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Librarys**

In [1]:
import pandas as pd
#Connections
from google.colab import drive

# **Connections**

In [2]:
#Google drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Loading datasets**

In [3]:
stores_file_path = '/content/drive/My Drive/DTM15/Dataset/Retail_salses_forecast/stores_data_set.csv'
sales_file_path = '/content/drive/My Drive/DTM15/Dataset/Retail_salses_forecast/sales_data_set.csv'
Features_file_path = '/content/drive/My Drive/DTM15/Dataset/Retail_salses_forecast/Features_data_set.csv'
stores_df = pd.read_csv(stores_file_path)
sales_df = pd.read_csv(sales_file_path)
features_df = pd.read_csv(Features_file_path)

# **stores**

In [4]:
stores_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [5]:
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


# **sales**

In [6]:
sales_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


In [7]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [8]:
sales_df['Date'] = pd.to_datetime(sales_df['Date'], format='%d/%m/%Y')

# **features**

In [9]:
features_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [10]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [11]:
features_df.isnull().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [15]:
features_df['Date'] = pd.to_datetime(features_df['Date'], format='%d/%m/%Y')

In [18]:
features_df.MarkDown1.fillna(0, inplace = True)
features_df.MarkDown2.fillna(0, inplace = True)
features_df.MarkDown3.fillna(0, inplace = True)
features_df.MarkDown4.fillna(0, inplace = True)
features_df.MarkDown5.fillna(0, inplace = True)

In [19]:
features_df.isnull().sum()

Store             0
Date              0
Temperature       0
Fuel_Price        0
MarkDown1         0
MarkDown2         0
MarkDown3         0
MarkDown4         0
MarkDown5         0
CPI             585
Unemployment    585
IsHoliday         0
dtype: int64

# **query**

**Which store performs high sales in week wise**

In [23]:
# Extracting week from the 'Date' column
sales_df['Week'] = sales_df['Date'].dt.strftime('%U')

# Grouping by store and week, then calculating the sum of weekly sales
weekly_sales_by_store = sales_df.groupby(['Store', 'Week'])['Weekly_Sales'].sum().reset_index()

# Finding the store with the highest weekly sales
store_with_highest_sales = weekly_sales_by_store.loc[weekly_sales_by_store.groupby('Week')['Weekly_Sales'].idxmax()].reset_index()

# Displaying the store with the highest sales in each week
print(store_with_highest_sales)


    index  Store Week  Weekly_Sales
0     156      4   01    3910242.34
1     157      4   02    3807179.07
2     158      4   03    3891491.70
3     159      4   04    3742961.36
4     992     20   05    6956061.74
5     993     20   06    6783474.32
6     162      4   07    6793995.99
7     163      4   08    6230485.70
8     996     20   09    6411461.09
9     997     20   10    6141173.01
10    166      4   11    6228872.18
11    687     14   12    5823295.21
12   1000     20   13    6408110.05
13    689     14   14    6672602.42
14   1002     20   15    6088586.98
15    691     14   16    6270292.74
16    692     14   17    5797002.71
17    693     14   18    6415070.74
18   1006     20   19    6238057.26
19    175      4   20    6126530.89
20    696     14   21    6338716.01
21   1009     20   22    6528992.63
22    698     14   23    6481631.92
23   1011     20   24    6322255.19
24    648     13   25    6073876.33
25   1013     20   26    6252794.79
26   1014     20   27    658

Identify average customer visit in the type B store in April Months

In [25]:
# Assuming sales_df contains the sales data
# First, convert the 'Date' column to datetime if it's not already
sales_df['Date'] = pd.to_datetime(sales_df['Date'])

# Filter sales data for Type B stores
type_b_stores_sales = sales_df[sales_df['Store'].isin(stores_df[stores_df['Type'] == 'B']['Store'])]

# Filter sales data for April
april_sales = type_b_stores_sales[(type_b_stores_sales['Date'].dt.month == 4)]

# Calculate the average customer visit
average_customer_visit = april_sales.groupby('Store')['Date'].nunique().mean()

print("Average customer visit in Type B stores in April:", average_customer_visit)


Average customer visit in Type B stores in April: 14.0


Identify best average sales in holiday week for all store types

In [26]:
# Filter sales data for holiday weeks
holiday_sales = sales_df[sales_df['IsHoliday']]

# Calculate total sales for each store during holiday weeks
total_sales_by_store = holiday_sales.groupby('Store')['Weekly_Sales'].sum()

# Merge total sales with store type
merged_sales = total_sales_by_store.reset_index().merge(stores_df, on='Store')

# Calculate average sales for each store type during holiday weeks
average_sales_by_type = merged_sales.groupby('Type')['Weekly_Sales'].mean()

# Identify store type with the highest average sales during holiday weeks
best_store_type = average_sales_by_type.idxmax()
best_average_sales = average_sales_by_type.max()

print("Best average sales in holiday week for all store types:", best_store_type)
print("Average sales value:", best_average_sales)

Best average sales in holiday week for all store types: A
Average sales value: 14700127.643636364


Which store had a worst sales in leap year

In [27]:
# Assuming sales_df contains the sales data
# First, convert the 'Date' column to datetime if it's not already
sales_df['Date'] = pd.to_datetime(sales_df['Date'])

# Filter sales data for the leap year
leap_year_sales = sales_df[sales_df['Date'].dt.year % 4 == 0]

# Calculate total sales for each store during the leap year
total_sales_by_store = leap_year_sales.groupby('Store')['Weekly_Sales'].sum()

# Find the store with the minimum total sales
worst_store = total_sales_by_store.idxmin()
worst_sales = total_sales_by_store.min()

print("Store with the worst sales in the leap year:", worst_store)
print("Total sales:", worst_sales)


Store with the worst sales in the leap year: 33
Total sales: 11435551.03


What is the expected sales of each department when unemployment factor is greater>8

In [28]:
# Assuming sales_df contains the sales data
# Assuming features_df contains the features data including unemployment factor

# Merge sales data with features data on the 'Store' and 'Date' columns
merged_data = pd.merge(sales_df, features_df, on=['Store', 'Date'])

# Filter data where unemployment factor is greater than 8
high_unemployment_sales = merged_data[merged_data['Unemployment'] > 8]

# Calculate average sales for each department
average_sales_by_department = high_unemployment_sales.groupby('Dept')['Weekly_Sales'].mean()

print("Expected sales of each department when unemployment factor is greater than 8:")
print(average_sales_by_department)


Expected sales of each department when unemployment factor is greater than 8:
Dept
1     17719.577853
2     43370.044649
3     10984.784400
4     25040.236272
5     19914.917085
          ...     
95    68200.568373
96    15487.837565
97    13024.744137
98     6225.629719
99      342.380187
Name: Weekly_Sales, Length: 81, dtype: float64


Aggregate the net(total) sales of each department on month wise

In [29]:
# Assuming sales_df contains the sales data
# Assuming the 'Date' column is already in datetime format

# Extract month from the 'Date' column
sales_df['Month'] = sales_df['Date'].dt.month

# Group by department and month, then calculate the sum of weekly sales
monthly_sales_by_department = sales_df.groupby(['Dept', 'Month'])['Weekly_Sales'].sum().reset_index()

print("Net (total) sales of each department on month-wise basis:")
print(monthly_sales_by_department)


Net (total) sales of each department on month-wise basis:
     Dept  Month  Weekly_Sales
0       1      1    4919530.75
1       1      2   12657329.37
2       1      3   10809758.44
3       1      4   17620651.88
4       1      5    7650736.26
..    ...    ...           ...
957    99      8      18816.56
958    99      9       4878.76
959    99     10      13665.09
960    99     11      78254.54
961    99     12     152432.68

[962 rows x 3 columns]


Identify better department performance based on the store on all the week

In [30]:
# Group by store and department, then calculate the sum of weekly sales
total_sales_by_store_department = sales_df.groupby(['Store', 'Dept'])['Weekly_Sales'].sum().reset_index()

# Find the department with the highest total sales for each store
best_department_by_store = total_sales_by_store_department.loc[total_sales_by_store_department.groupby('Store')['Weekly_Sales'].idxmax()]

print("Department performance based on the store across all weeks:")
print(best_department_by_store)


Department performance based on the store across all weeks:
      Store  Dept  Weekly_Sales
69        1    92   19370632.64
147       2    92   23572153.03
190       3    38   15529566.07
297       4    92   22789210.43
340       5    38    7893570.24
446       6    92   14160545.90
509       7    72    6447844.96
601       8    95    9002059.31
641       9    38   11184287.78
736      10    72   20410926.56
829      11    95   11080680.91
835      12     2   10652763.01
980      13    92   23170876.20
1057     14    92   26101497.71
1101     15    38    7517821.66
1179     16    38    7447254.23
1255     17    38   11059588.03
1331     18    38    9339684.03
1445     19    92   16261990.45
1523     20    92   23542625.04
1532     21     2    6832625.68
1643     22    38   10015536.66
1740     23    72   10988695.25
1830     24    92   17429136.57
1874     25    38    6381753.99
1950     26    38   14800243.10
2061     27    92   20952094.22
2138     28    92   14083635.33
2181     29 